# Celeba df

In [2]:
import os
import pandas as pd
import numpy as np

In [3]:
with open("list_attr_celeba.txt", 'r') as attr_file:
    attr_file.readline()
    with open("list_attr_celeba.csv", 'w') as new_attr_file:
        new_line = "idx,filename,"
        for e in attr_file.readline().split():
            new_line += e + ','
        new_line = new_line[:-1] + "\n"
        new_attr_file.write(new_line)
        for k, line in enumerate(attr_file.readlines()):
            new_line = f"{k},"
            for e in line.split():
                new_line += e + ','
            new_line = new_line[:-1] + "\n"
            new_attr_file.write(new_line)


In [4]:
# 202599
df = pd.read_csv("list_attr_celeba.csv", header=0)
df["idx"] = pd.to_numeric(df["idx"])
KEY = "idx"
df.set_index(KEY, inplace=True)
ATTRIBUTES = df.columns[1:]
print(ATTRIBUTES)
for att in ATTRIBUTES:
    df[att] = df[att] == 1

Index(['5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive', 'Bags_Under_Eyes',
       'Bald', 'Bangs', 'Big_Lips', 'Big_Nose', 'Black_Hair', 'Blond_Hair',
       'Blurry', 'Brown_Hair', 'Bushy_Eyebrows', 'Chubby', 'Double_Chin',
       'Eyeglasses', 'Goatee', 'Gray_Hair', 'Heavy_Makeup', 'High_Cheekbones',
       'Male', 'Mouth_Slightly_Open', 'Mustache', 'Narrow_Eyes', 'No_Beard',
       'Oval_Face', 'Pale_Skin', 'Pointy_Nose', 'Receding_Hairline',
       'Rosy_Cheeks', 'Sideburns', 'Smiling', 'Straight_Hair', 'Wavy_Hair',
       'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick',
       'Wearing_Necklace', 'Wearing_Necktie', 'Young'],
      dtype='object')


In [5]:
df.head()

,filename,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
idx,,,,,,,,,,,,,,,,,,,,,
0,000001.jpg,False,True,True,False,False,False,False,False,False,...,False,True,True,False,True,False,True,False,False,True
1,000002.jpg,False,False,False,True,False,False,False,True,False,...,False,True,False,False,False,False,False,False,False,True
2,000003.jpg,False,False,False,False,False,False,True,False,False,...,False,False,False,True,False,False,False,False,False,True
3,000004.jpg,False,False,True,False,False,False,False,False,False,...,False,False,True,False,True,False,True,True,False,True
4,000005.jpg,False,True,True,False,False,False,True,False,False,...,False,False,False,False,False,False,True,False,False,True


# e4e model

In [ ]:
import os
import sys
sys.path.append(os.path.join(os.path.realpath(os.curdir), "encoder4editing"))

import torch
import torchvision.transforms as transforms
import pickle

from argparse import Namespace
from PIL import Image

from utils.common import tensor2im
from models.psp import pSp_encoder  # we use the pSp framework to load the e4e encoder.

import dlib
from utils.alignment import align_face

MODEL_PATH = "encoder4editing/pretrained_models/e4e_ffhq_encode.pt"
RESIZE_DIMS = (256, 256)
SHAPE_PREDICTOR_PATH = "shape_predictor_68_face_landmarks.dat"

In [ ]:
ckpt = torch.load(MODEL_PATH, map_location='cpu')
opts = ckpt['opts']
# pprint.pprint(opts)  # Display full options used
# update the training options
opts['checkpoint_path'] = MODEL_PATH
opts= Namespace(**opts)
net = pSp_encoder(opts)
net.eval()
net.cuda()
print('Model successfully loaded!')


transform_fn = transforms.Compose([
    transforms.Resize(RESIZE_DIMS),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])]"")

def run_alignment(image_path):
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    aligned_image = align_face(filepath=image_path, predictor=predictor) 
    return aligned_image 


def run_on_batch(inputs, net):
    images, latents = net(inputs.to("cuda").float(), randomize_noise=False, return_latents=True)
    return images, latents

In [ ]:
all_latents = dict()

for i in df.index:
    img_path = df.loc[i]["filename"]
    img = run_alignment(img_path)
    img = transform_fn(img)
    img, latents = run_on_batch(img.unsqueeze(0), net)
    all_latents[img_path] = latents


In [ ]:
with open("all_latents", "wb") as all_latents_file:
    pickle.dump(all_latents)


# Latent operations

In [16]:
import torch
import pickle
import matplotlib.pyplot as plt

with open("all_latents_numpy", "rb") as all_latents_file:
    all_latents = pickle.load(all_latents_file)

print("nb latents computed:", len(all_latents))

nb latents computed: 39001


In [17]:
print(all_latents["img_align_celeba/000001.jpg"])

[[[ 0.58281183  0.19550136  0.9119964  ...  0.00978688 -0.11105777
   -0.04303272]
  [ 0.31136888  0.9483163   1.2595953  ... -0.01400552 -0.2718999
    0.21017043]
  [ 1.8234559   0.950791    0.3838197  ...  0.03282372  0.04306225
    0.12083358]
  ...
  [ 0.6397143   0.1256532   0.9169841  ...  0.20252512 -0.24869095
   -0.05217373]
  [ 0.5856341   0.1937655   0.9148084  ...  0.01347185 -0.10989983
   -0.0440767 ]
  [ 0.52293086  0.21180582  0.9762995  ...  0.15644972 -0.40757364
   -0.20369315]]]


In [18]:
df_latents = df.iloc[:len(all_latents)]

In [19]:
print(list(all_latents.keys())[40:60])

['img_align_celeba/000041.jpg', 'img_align_celeba/000042.jpg', 'img_align_celeba/000043.jpg', 'img_align_celeba/000044.jpg', 'img_align_celeba/000045.jpg', 'img_align_celeba/000046.jpg', 'img_align_celeba/000047.jpg', 'img_align_celeba/000048.jpg', 'img_align_celeba/000049.jpg', 'img_align_celeba/000050.jpg', 'img_align_celeba/000051.jpg', 'img_align_celeba/000052.jpg', 'img_align_celeba/000053.jpg', 'img_align_celeba/000054.jpg', 'img_align_celeba/000055.jpg', 'img_align_celeba/000056.jpg', 'img_align_celeba/000057.jpg', 'img_align_celeba/000058.jpg', 'img_align_celeba/000059.jpg', 'img_align_celeba/000060.jpg']


In [33]:
def get_att_df(df, att_name, yes=True):
    if yes:
        res = df[df[att_name]]
    else:
        res = df[df[att_name] == False]
    return res


def get_latents(df, **kwargs):
    for att in ATTRIBUTES:
        if att in kwargs:
            df = get_att_df(df, att, kwargs[att])

    att_latents = np.concatenate([all_latents["img_align_celeba/"+n] for n in df["filename"]], 0)
    return att_latents

print(get_latents(df_latents, **{"Bald": True, "Male": False}))

         filename  5_o_Clock_Shadow  Arched_Eyebrows  Attractive  \
idx                                                                
10648  010649.jpg             False             True       False   
31675  031676.jpg             False             True       False   

       Bags_Under_Eyes  Bald  Bangs  Big_Lips  Big_Nose  Black_Hair  ...  \
idx                                                                  ...   
10648            False  True  False      True     False       False  ...   
31675            False  True  False     False     False       False  ...   

       Sideburns  Smiling  Straight_Hair  Wavy_Hair  Wearing_Earrings  \
idx                                                                     
10648      False    False          False      False              True   
31675      False    False          False      False              True   

       Wearing_Hat  Wearing_Lipstick  Wearing_Necklace  Wearing_Necktie  Young  
idx                                             

In [34]:
import os
import tkinter as tk
import torch
from PIL import Image, ImageTk


class ImgCanvas(tk.Canvas):
    def __init__(self, root, width=256, height=512):
        super().__init__(root, width=width, height=height)
        self.width = width
        self.height = height
        self.create_line(2, 0, 2, height)

        self.pack(side=tk.RIGHT, fill=tk.Y)

    def draw_image(self, img, original=False):
        img = ImageTk.PhotoImage(img.resize((self.width, self.height//2)))
        if original:
            y = self.height // 2
            self.original_img = img
            self.create_image(0, y, anchor="nw", image=self.original_img)
        else:
            y = 0
            self.edit_img = img
            self.create_image(0, y, anchor="nw", image=self.edit_img)
        

class PathFrame(tk.Frame):
    def __init__(self, root, entry_callback, width=700, height=50):
        super().__init__(root, width=width, height=height)
        self.width = width
        self.height = height
        
        self.entry_callback = entry_callback

        self.path = tk.StringVar(value="Enter image path")
        self.entry = tk.Entry(self, textvariable=self.path, width=50)
        self.entry.bind("<Return>", lambda e: self.entry_callback())
        self.entry.place(x=5, y=5)

        self.button = tk.Button(self, text="Load", command=self.entry_callback)
        self.button.place(x=320, y=1)

        self.result = tk.Label(self)
        self.result.place(x=5, y=25)

        self.pack(side=tk.TOP)


In [ ]:
class DeltaScaleFrame(tk.Frame):
    def __init__(self, root, edit_command):
        super().__init__(root)
        self.edit_command = edit_command
        self.scale_bar = tk.Scale(self, orient='horizontal', from_=-2, to=2, command=self.edit_command,
                                  resolution=0.02, tickinterval=10, width=10, length=580)
        self.scale_bar.pack(tk.TOP)
        self.pack(tk.TOP)


class AttributesLabelsFrame(tk.Frame):
    def __init__(self, root, width=150, height=512):
        super().__init__(root, width=width, height=height)
        self.labels = []
        for att in ATTRIBUTES:
            self.labels.append(tk.Label(self, text=att))
            self.labels[-1].pack(tk.TOP)
        self.pack(tk.LEFT)


class GroupFrame(tk.Frame):
    def __init__(self, root, width=150, height=512)
        super().__init__(root, width=width, height=height)
        self.att_frames = []
        for att in ATTRIBUTES:
            self.att_frames.append({"frame": tk.Frame(self, width=150, height=10)})
            self.att_frames[-1]["var"] = tk.IntVar(value=-1)
            self.att_frames[-1]["bt1"] = tk.Radiobutton(self.att_frames[-1]["frame"], value=-1, variable=self.att_frames[-1]["var"])
            self.att_frames[-1]["bt2"] = tk.Radiobutton(self.att_frames[-1]["frame"], value=1, variable=self.att_frames[-1]["var"])
            self.att_frames[-1]["bt3"] = tk.Radiobutton(self.att_frames[-1]["frame"], value=0, variable=self.att_frames[-1]["var"])
            self.att_frames[-1]["bt1"].pack(tk.LEFT)
            self.att_frames[-1]["bt2"].pack(tk.LEFT)
            self.att_frames[-1]["bt3"].pack(tk.LEFT)
    
    def get_dict(self):
        for i, att in enumerate(ATTRIBUTES):


class CelebaConfigFrame(tk.Frame):
    def __init__(self, root):
        super().__init__(root)
        self.delta_scale = DeltaScaleFrame(self)
        self.attributes_labels = AttributesLabelsFrame(self)
        self.group1 = GroupFrame(self)
        self.group2 = GroupFrame(self)

class CelebaEditWindow(tk.Tk):
    def __init__(self):
        self.img_frame = ImgCanvas(self, width=256, height=512)
        self.path_frame = PathFrame(self)
        self.config_frame = CelebaConfigFrame(self)